In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

In [2]:
SHOW_FIGURES = True

In [3]:
if not SHOW_FIGURES:
    matplotlib.use("pgf")
    matplotlib.rcParams.update({
        "pgf.texsystem": "pdflatex",
        'font.family': 'serif',
        'text.usetex': True,
        'pgf.rcfonts': False,
        'text.latex.preamble' : r'\newcommand{\mathdefault}[1][]{}'
    })


In [4]:
file_path = '../merged.csv'
file_path = '../results_Hungarian/results.csv'
#column_names = ['delta', 'state', 'time', 'occupation', 'fairness']
#df = pd.read_csv(file_path)

In [5]:
csv_dict = []
with open(file_path, 'r') as file:
    # Iterate over each line in the fileplt.plot(x_fit, y_fit, color='green', label='Polynomial Approximation')
    for line in file:
        # Split the line content by comma
        line_data = line.strip().split(',')

        if len(line_data) == 7:
            seven_keys = ['delta', 'state', 'time', 'occupation', 'fairness', 'allocated', 'skipped']
            csv_dict.append(dict(zip(seven_keys, line_data)))

        elif len(line_data) == 15:
            fifteen_keys = ['method', 'execution time', 'delta', 'average make span', 'result message',
                            'makespan', 'resource occupation', 'resource fairness',
                            'assigned tasks', 'allocated tasks', 'postponed tasks',
                            'optimal solution', 'feasible solution', 'no solution',
                            'resource worklist rule'
                            ]
            csv_dict.append(dict(zip(fifteen_keys, line_data)))
        elif len(line_data) == 19:
            nineteen_keys = ['method', 'start time', 'end time', 'real execution time', 'system execution time',
                             'delta', 'average make span', 'result message',
                            'makespan', 'resource occupation', 'resource fairness', 'weighted resource fairness',
                            'assigned tasks', 'allocated tasks', 'postponed tasks',
                            'optimal solution', 'feasible solution', 'no solution',
                            'resource worklist rule'
                            ]
            row_dict = dict(zip(nineteen_keys, line_data))
            for k in ['optimal solution', 'feasible solution', 'no solution']:
                if row_dict[k] == '':
                    row_dict[k] = 0
            csv_dict.append(row_dict)
        else:
            print(len(line_data))

        #print(line_data)
        # Print or process the split data as needed
        #print(line_data)
df_dtypes = {'method' : str, 'start time' : str, 'end time' : str, 'real execution time' : float, 'system execution time' : float,
             'delta' : float, 'makespan' : float, 'result message' : str,
             'makespan' : float, 'resource occupation' : float, 'resource fairness' : float, 'weighted resource fairness' : float,
             'assigned tasks' : int, 'allocated tasks' : int, 'postponed tasks' : int,
             'optimal solution' : int, 'feasible solution' : int, 'no solution' : int,
             'resource worklist rule' : str}
df = pd.DataFrame(csv_dict)
for k, v in df_dtypes.items():
    df[k] = df[k].astype(v)

FileNotFoundError: [Errno 2] No such file or directory: '../results_Hungarian/results.csv'

In [ ]:
df = df[df['method'] == 'Hungarian']
df.reset_index(inplace=True)
df

In [ ]:
# Df has some values like delta=1.99
# Filter so that distances are 0.02 e.g. [..., 1.94, 1.96, 1.98, 2.0]
n = 1000
df = df[((df['delta']*n).astype(int) % int(0.02 * n)) == 0]
df

In [ ]:
# When two equal delta values are present sample a random one
result = df.groupby('delta').apply(lambda x: x.sample(1)).reset_index(drop=True)
#result
df = result

In [ ]:
df

In [ ]:
from scipy.optimize import curve_fit

def polynomial_func(x, *coeffs):
    return sum(c * x**i for i, c in enumerate(coeffs))

degree = 2  # Adjust the degree of the polynomial
params, _ = curve_fit(polynomial_func, df['delta'], df['makespan'], p0=[1] * (degree + 1))

x_fit = np.linspace(df['delta'].min(), df['delta'].max(), 100)
y_fit = polynomial_func(x_fit, *params)



In [ ]:
df = df[df['result message'] != 'Stopped']
df = df.sort_values(by='delta')

In [ ]:
def plot(x, y, labels=None, colors=plt.rcParams["axes.prop_cycle"](), degree=2, yrange=None):
    plt.clf()
    fig = plt.gcf()
    fig.set_size_inches(6, 2.5)
    
    # Plot the data
    p = plt.scatter(x, y, marker='o', color=next(colors)['color'])
    
    # Set labels and title
    if labels:
        plt.xlabel(labels[0])
        plt.ylabel(labels[1])
    else:
        plt.xlabel('x')
        plt.ylabel('y')

    params, _ = curve_fit(polynomial_func, x, y, p0=[1] * (degree + 1))
    
    x_fit = np.linspace(x.min(), x.max(), 100)
    y_fit = polynomial_func(x_fit, *params)
    plt.plot(x_fit, y_fit, color=p.get_facecolor(), label='Polynomial Approximation')

    if yrange:
        plt.ylim(yrange[0], yrange[1])
        plt.yticks(np.linspace(yrange[0], yrange[1], 11))
    else:
        y_range = y.max() - y.min()
        plt.ylim(y.min() - 0.1*y_range, y.max() + 0.1*y_range)
        plt.yticks(np.linspace(y.min(), y.max(), 10))
    plt.xticks(np.linspace(0.9, 2.0, 12))

    plt.xlim(0.85, 2.05)
    
    plt.xticks(rotation=45)
    plt.tight_layout(pad=0.2)
    
    # Show the plot
    if  not SHOW_FIGURES:
        plt.savefig('task_assignment_' + labels[0] + '_' + labels[1] + '.pgf')
    plt.show()


colors = plt.rcParams["axes.prop_cycle"]()

plot(df['delta'], df['makespan'], ('Delta', 'Avg. cycle time'), yrange=(10, 25), colors=colors)
plot(df['delta'], df['resource occupation'], ('Delta', 'Resource occupation'), colors=colors, yrange=(0.5,1))
plot(df['delta'], df['resource fairness'], ('Delta', 'Resource fairness'), colors=colors, yrange=(0.0,0.1))
plot(df['delta'], df['weighted resource fairness'], ('Delta', 'Weighted resource fairness'), colors=colors, yrange=(0.5,3.5))
plot(df['delta'], df['postponed tasks'], ('Delta', 'Postponed tasks'), colors=colors)
plot(df['delta'], df['allocated tasks'], ('Delta', 'Allocated tasks'), colors=colors)
plot(df['delta'], df['allocated tasks'], ('Delta', 'Assigned tasks'), colors=colors)